In [2]:
import os 
import copy
import json
import numpy as np
import torch
import torch.nn as nn
from PIL import Image
import datasets
from datasets import load_dataset,load_from_disk
from datasets import Dataset, DatasetDict

from transformers import AutoTokenizer
from modeling_layoutxlm_re import LayoutLMv2ForRelationExtraction

In [3]:
DP = "/mnt/j/dataset/document-intelligence/XFUND/fr"
OP = "/mnt/j/dataset/document-intelligence/XFUND/output"
MP = "/mnt/j/model/pretrained-model/bert_torch"
TRAINED_MP = "./output/layoutxlm-finetuned-xfund-re/"

use_visual_backbone=False

#tokenizer = AutoTokenizer.from_pretrained(os.path.join(MP, "microsoft-layoutxlm-base"))
tokenizer = AutoTokenizer.from_pretrained(os.path.join(TRAINED_MP, "checkpoint-5000"))
#model = LayoutLMv2ForRelationExtraction.from_pretrained(os.path.join(MP, "microsoft-layoutxlm-base"),use_visual_backbone=use_visual_backbone)
# re训练的模型
model = LayoutLMv2ForRelationExtraction.from_pretrained(os.path.join(TRAINED_MP, "checkpoint-5000"),use_visual_backbone=use_visual_backbone)



use_visual_backbone: False


In [4]:

train_dataset = Dataset.from_file(os.path.join(DP, "fr.train.arrow"))
test_dataset = Dataset.from_file(os.path.join(DP, "fr.val.arrow"))
dataset = DatasetDict({"train": train_dataset, "validation": test_dataset})
print("dataset:", dataset)

dataset: DatasetDict({
    train: Dataset({
        features: ['id', 'input_ids', 'bbox', 'labels', 'image', 'original_image', 'entities', 'relations'],
        num_rows: 202
    })
    validation: Dataset({
        features: ['id', 'input_ids', 'bbox', 'labels', 'image', 'original_image', 'entities', 'relations'],
        num_rows: 71
    })
})


In [6]:
example = dataset['train'][0]
print(example.keys())
# print(example)
print("example['input_ids']:",example['input_ids'])
print("example['input_ids'] len:",len(example['input_ids']))

dict_keys(['id', 'input_ids', 'bbox', 'labels', 'image', 'original_image', 'entities', 'relations'])
example['input_ids']: [72791, 2149, 104, 25, 101287, 990, 106, 22364, 61038, 35213, 1514, 335, 76783, 11338, 109893, 122112, 84780, 31244, 51552, 441, 132824, 35213, 14614, 167795, 618, 89097, 85621, 7, 62, 5, 61038, 35213, 238530, 40713, 839, 80020, 615, 55448, 628, 38016, 149896, 13, 8, 21, 17752, 22, 6, 102250, 115, 19183, 94228, 531, 104067, 12, 438, 39, 6470, 68915, 170158, 6, 58745, 14154, 60221, 166132, 4, 436, 187367, 17166, 7, 628, 6, 138971, 64, 54601, 1830, 628, 123708, 64, 166697, 13, 628, 5054, 9942, 384, 50763, 26551, 132, 7, 16, 6, 58745, 4880, 192508, 196417, 3768, 72813, 25388, 148573, 15, 112, 96, 25, 165264, 16, 6, 191712, 981, 84624, 5, 277, 20897, 64, 88154, 64, 22469, 38016, 48218, 1426, 5, 191712, 5, 277, 787, 2041, 2312, 80273, 13, 104, 25, 40098, 12, 15, 16305, 27227, 104, 25, 309, 1193, 21, 108090, 42518, 1236, 253, 2771, 49894, 16, 6, 166697, 13, 20, 6, 179149

In [5]:
# https://blog.csdn.net/gjj19901005/article/details/127530571

# dummy_model_input = tokenizer(["This is a sample"], return_tensors=None)
 
# dummy_model_input['input_ids'] = torch.zeros((1,512),dtype=torch.int64) + 10
# dummy_model_input['bbox'] = torch.zeros((1,512, 4), dtype=torch.int64) + 20
# dummy_model_input['attention_mask'] = torch.zeros((1,512), dtype=torch.int64) + 1
# dummy_model_input['start'] = torch.tensor(np.asarray([[0,3,5,7,9,-1]], dtype=np.int64).reshape((1, -1)))
# # dummy_model_input['end'] = torch.tensor(np.asarray([3,5,7,9,15], dtype=np.int64).reshape((1, -1)))
# dummy_model_input['labels'] = torch.tensor(np.asarray([[1, 2, 1, 2, 1]], dtype=np.int64).reshape((1, -1)))
# dummy_model_input['head'] = torch.tensor(np.asarray([[0,2,4]], dtype=np.int64).reshape((1, -1)))
# dummy_model_input['tail'] = torch.tensor(np.asarray([[1,3,3]], dtype=np.int64).reshape((1, -1)))


# print(dummy_model_input)

In [6]:
# model.eval()
# torch.onnx.export(
#     model, 
#     (dummy_model_input['input_ids'], 
#     dummy_model_input['bbox'], 
#     dummy_model_input['attention_mask'], 
#     dummy_model_input['start'],
#     # dummy_model_input['end'],
#     dummy_model_input['labels'],
#     dummy_model_input['head'],
#     dummy_model_input['tail']),
#     f="./layoutxlm_re.onnx",  
#     input_names=['input_ids','bbox','attention_mask', 'start', 'labels', 'head', 'tail'], 
#     output_names=['logits'], 
#     dynamic_axes={'input_ids': {0: 'batch_size'}, 
#                   'bbox': {0: 'batch_size'},
#                   'attention_mask': {0: 'batch_size'}, 
#                   'start': {0: 'batch_size', 1: 'entity_len'}, 
#                 #   'end': {0: 'batch_size', 1: 'entity_len'},
#                   'labels': {0: 'batch_size', 1: 'entity_len'}, 
#                   'head': {0: 'batch_size', 1: 'relation_len'},
#                   'tail': {0: 'batch_size', 1: 'relation_len'}, 
#                   'logits':{0: 'batch_size', 1:'relation_len'}
#                   }, 
#     # do_constant_folding=True, 
#     # verbose=True,
#     opset_version=13, 
# )

In [8]:
XFUND_JSON = "/mnt/j/dataset/document-intelligence/XFUND/fr"
image_path = "/mnt/j/dataset/document-intelligence/XFUND/fr/fr.val"

from utils.image_utils import *

label2id = {"HEADER":0, "QUESTION":1, "ANSWER":2}

with open(os.path.join(XFUND_JSON, "fr.val.json"), encoding="utf-8", mode="r") as fi:
    ann_infos = json.load(fi)
    document_list = ann_infos["documents"]
    tokenizer = AutoTokenizer.from_pretrained(os.path.join(MP, "xlm-roberta-base"))

for guid, doc, in enumerate(document_list):
    image_file = os.path.join(image_path, doc["img"]["fname"])
    print(image_file)
    size = [doc["img"]["width"], doc["img"]["height"]]
    image, size = load_image(image_file, size=224)
    original_image, _ = load_image(image_file)
    document = doc["document"]
    tokenized_doc = {"input_ids": [], "bbox": [], "labels": []}
    entities = []
    relations = []
    id2label = {}
    entity_id_to_index_map = {}
    empty_entity = set()
    for line in document:
        if len(line["text"]) == 0:
            empty_entity.add(line["id"])
            continue
        id2label[line["id"]] = line["label"]
        relations.extend([tuple(sorted(l)) for l in line["linking"]])
        # 将文本转为id
        # offset_mapping：每一个token具体占了几个char，中文都是一个char，英文就看单词的长度， [PAD]全都是0
        tokenized_inputs = tokenizer(
            line["text"],
            add_special_tokens=False,
            return_offsets_mapping=True,
            return_attention_mask=False,
        )
        text_length = 0
        ocr_length = 0
        bbox = []
        for token_id, offset in zip(tokenized_inputs["input_ids"], tokenized_inputs["offset_mapping"]):
            if token_id == 6:
                bbox.append(None)
                continue
            text_length += offset[1] - offset[0]
            tmp_box = []
            while ocr_length < text_length:
                ocr_word = line["words"].pop(0)
                ocr_length += len(
                    tokenizer._tokenizer.normalizer.normalize_str(ocr_word["text"].strip())
                )
                tmp_box.append(simplify_bbox(ocr_word["box"]))
            if len(tmp_box) == 0:
                tmp_box = last_box
            bbox.append(normalize_bbox(merge_bbox(tmp_box), size))
            last_box = tmp_box  # noqa
        #
        bbox = [
            [bbox[i + 1][0], bbox[i + 1][1], bbox[i + 1][0], bbox[i + 1][1]] if b is None else b
            for i, b in enumerate(bbox)
        ]
        # 生成每个字或者词的标签
        if line["label"] == "other":
            label = ["O"] * len(bbox)
        else:
            label = [f"I-{line['label'].upper()}"] * len(bbox)
            label[0] = f"B-{line['label'].upper()}"
        tokenized_inputs.update({"bbox": bbox, "labels": label})
        if label[0] != "O":
            entity_id_to_index_map[line["id"]] = len(entities)
            entities.append(
                {
                    "start": len(tokenized_doc["input_ids"]),
                    "end": len(tokenized_doc["input_ids"]) + len(tokenized_inputs["input_ids"]),
                    "label": line["label"].upper(),
                }
            )
        for i in tokenized_doc:
            tokenized_doc[i] = tokenized_doc[i] + tokenized_inputs[i]
    relations = list(set(relations))
    relations = [rel for rel in relations if rel[0] not in empty_entity and rel[1] not in empty_entity]
    kvrelations = []
    for rel in relations:
        pair = [id2label[rel[0]], id2label[rel[1]]]
        if pair == ["question", "answer"]:
            kvrelations.append(
                {"head": entity_id_to_index_map[rel[0]], "tail": entity_id_to_index_map[rel[1]]}
            )
        elif pair == ["answer", "question"]:
            kvrelations.append(
                {"head": entity_id_to_index_map[rel[1]], "tail": entity_id_to_index_map[rel[0]]}
            )
        else:
            continue

    def get_relation_span(rel):
        bound = []
        for entity_index in [rel["head"], rel["tail"]]:
            bound.append(entities[entity_index]["start"])
            bound.append(entities[entity_index]["end"])
        return min(bound), max(bound)

    relations = sorted(
        [
            {
                "head": rel["head"],
                "tail": rel["tail"],
                "start_index": get_relation_span(rel)[0],
                "end_index": get_relation_span(rel)[1],
            }
            for rel in kvrelations
        ],
        key=lambda x: x["head"],
    )
    chunk_size = 512
    for chunk_id, index in enumerate(range(0, len(tokenized_doc["input_ids"]), chunk_size)):
        item = {}
        for k in tokenized_doc:
            item[k] = tokenized_doc[k][index: index + chunk_size]
        entities_in_this_span = []  # 把相同的实体合并在一起?
        global_to_local_map = {}
        for entity_id, entity in enumerate(entities):
            if (
                    index <= entity["start"] < index + chunk_size
                    and index <= entity["end"] < index + chunk_size
            ):
                entity["start"] = entity["start"] - index
                entity["end"] = entity["end"] - index
                global_to_local_map[entity_id] = len(entities_in_this_span)
                entities_in_this_span.append(entity)
        relations_in_this_span = []
        for relation in relations:
            if (
                    index <= relation["start_index"] < index + chunk_size
                    and index <= relation["end_index"] < index + chunk_size
            ):
                relations_in_this_span.append(
                    {
                        "head": global_to_local_map[relation["head"]],
                        "tail": global_to_local_map[relation["tail"]],
                        "start_index": relation["start_index"] - index,
                        "end_index": relation["end_index"] - index,
                    }
                )
        item.update(
            {
                "id": f"{doc['id']}_{chunk_id}",
                "image": image,
                "original_image": original_image,
                "entities": entities_in_this_span,
                "relations": relations_in_this_span,
            }
        )
        print("item:", item)
        model_input = {}
        input_ids = [0]*512
        input_ids[0:len(item[k])] = item["input_ids"]
        model_input["input_ids"] = torch.LongTensor([input_ids])
        
        bbox = []
        for i in range(512):
            bbox.append([0,0,0,0])
        print("ccc:", bbox)
        bbox[0:len(item[k])] = item["bbox"]
        model_input["bbox"] = torch.LongTensor([bbox])
        
        attention_mask = [0]*512
        attention_mask[0:len(item[k])] = [1]*len(item[k])
        model_input["attention_mask"] = torch.FloatTensor([attention_mask])
        
        print("label2id:", label2id)
        entity_dict = {'start': [entity["start"] for entity in item["entities"]],
                        'end': [entity["end"] for entity in item["entities"]],
                        'label': [label2id[entity["label"]] for entity in item["entities"]]}
        print("entity_dict:", entity_dict)

        model_input["entities"] = [entity_dict]
        model_input["relations"] = [{'start_index': [], 'end_index': [], 'head': [], 'tail': []}]
        
        print("model_input:", model_input)
        print("input_ids shape:", model_input["input_ids"].shape)
        print("bbox shaep:", model_input["bbox"].shape)
        print("attention_mask shape:", model_input["attention_mask"].shape)
        
        with torch.no_grad():
            # inputs: ocr处理侯的输入，ids（SER和RE的inputs是一的）
            outputs = model(**model_input)
            print("output:", outputs)
        
       
    break
        

/mnt/j/dataset/document-intelligence/XFUND/fr/fr.val/fr_val_0.jpg
item: {'input_ids': [12490, 2609, 68819, 169, 741, 91870, 5849, 199, 110267, 7, 41, 1023, 81418, 163465, 578, 233404, 4178, 48522, 7, 15, 57595, 7, 4, 60525, 4, 149592, 4, 3021, 5, 194, 182946, 3190, 5, 12498, 16277, 19116, 9, 13048, 113575, 44918, 8, 96, 25, 284, 36737, 253, 51347, 152, 76106, 82, 118009, 8, 87, 25, 683, 6371, 32, 77035, 28636, 224, 5470, 7, 72054, 77035, 28636, 199, 28486, 7, 94505, 628, 104, 25, 19988, 831, 10442, 4, 115953, 42, 77035, 28636, 95, 333, 1606, 104, 25, 309, 1764, 4002, 15, 272, 68819, 169, 115953, 42, 16, 77035, 67362, 2921, 152, 572, 5, 125508, 16271, 84780, 845, 230998, 41, 199, 32565, 94228, 90, 807, 405, 123780, 82, 199, 60525, 1059, 9, 513, 4288, 7, 2045, 34913, 532, 19640, 82, 34913, 27430, 19736, 5, 821, 25, 28792, 3075, 95, 123054, 104, 25, 81317, 7, 253, 180150, 199, 32565, 94228, 90, 807, 95, 39397, 12937, 4, 113, 48402, 199, 32565, 141859, 7, 4, 82, 199, 138488, 82, 52193, 100